In [336]:
from scipy.stats import norm
from time import time
import random    
import pandas as pd

In [337]:
class cycliste:
    
    def log(self):
        try:
            print "logging",
            with open("./cyclistes.csv", "a") as f:
                df = pd.DataFrame(self.__dict__)
                f.write(df.to_csv())
                f.write("\n")
        except Exception as e:
            print e
            
        
    def __init__(self, identifiant):
        import time
        from scipy.stats import norm
        import random
        homme   = "homme"
        femme   = "femme"
        age_min = 15
        age_max = 80
        sexe    = random.choice([homme, femme])
        age     = random.randint(age_min ,age_max)
        sportif = random.choice([-0.5, 0, 2, 4, 6 ])
        if age < 35:
            facteur = norm( loc =1.2, scale=0.2 )
        elif age <50:
            facteur = norm( loc =1  , scale=0.2 )
        elif age <90:
            facteur = norm( loc =0.7, scale=0.2 )

        if sexe == homme:
            vitesse_de_base = 20
            nb_km = norm( loc =7, scale=2 )
        else:
            vitesse_de_base = 15
            nb_km = norm( loc =10, scale=2 )
        vitesse_moyenne = vitesse_de_base * facteur.rvs(1) + sportif
        
        self.sportif = sportif
        self.age = age
        self.sexe = sexe
        self.nb_km = nb_km.rvs(1)[0]
        self.vitesse = vitesse_moyenne[0]
        self.sur_velo =False
        self.debug = False
        self.nb_trajet = 0
        self.durees = []
        self.trajets = []
        self.velo = False
        self.identifiant = identifiant
        

    def prend_velo(self, station):
        if self.sur_velo:
            if self.debug : print "deja sur velo"
            pass
        else:
            import time
            velo = station.prendre_velo()
            if velo:
                velo.disponible      = False
                self.sur_velo        = True
                self.heure_de_depart = time.time()
                try:
                    nb_km_local          = norm( loc =self.nb_km, scale=self.nb_km/float(3) ).rvs(1)[0]
                except Exception as e:
                    print "pbm sur nb_km_local = self.nb_km = %s "
                    print e
                try:
                    vitesse_local        = norm(loc = self.vitesse, scale = self.vitesse/float(10)).rvs(1)[0]
                    vitesse_local        = vitesse_local * velo.performance
                except:
                    print "pbm sur vitesse_local"
                    print "self.vitesse = %s"%self.vitesse
                    vitesse_local = self.vitesse
                self.nb_km_trajet    = nb_km_local
                duree                = nb_km_local / vitesse_local
                if self.debug : print "arrivee dans %s a %s"%(duree, self.heure_d_arrivee)
                self.velo = velo
                
                self.heure_d_arrivee =  self.heure_de_depart + duree
                
                if velo.performance <0.1:
                    duree = 0
                    self.durees.append( duree)
                    self.heure_d_arrivee =  self.heure_de_depart
                    self.nb_km_trajet    = 0
                    self.rendre_velo(station)
                elif velo.performance <0.3:
                    duree = duree / 10
                    self.durees.append( duree)
                    self.heure_d_arrivee =  self.heure_de_depart + duree
                    self.nb_km_trajet    = self.nb_km_trajet / float(10)
                    self.rendre_velo(station)
                elif velo.performance <0.5:
                    duree = duree / 5
                    self.durees.append( duree)
                    self.nb_km_trajet    = self.nb_km_trajet / float(5)
                    self.heure_d_arrivee =  self.heure_de_depart + duree 
                else:
                    self.durees.append( duree)
                    
            else:
                if self.debug : print "pas de velo ici"


    def rendre_velo(self, station):        
        import time
        if not self.sur_velo:
            if self.debug : print "pas en chemin"
            return 0
        if time.time() > self.heure_d_arrivee:
            if station.rendre_velo(self.velo, self.nb_km_trajet):
                if self.debug : print "plus sur velo"
                duree_constatee = self.heure_d_arrivee - self.heure_de_depart
                self.trajets.append((self.heure_de_depart, self.heure_d_arrivee, self.nb_km_trajet, self.velo.nom ))
                self.sur_velo =False
                self.nb_trajet = self.nb_trajet + 1
                self.velo = False
            else:
                if self.debug : print "impossible de rendre sur station %s"%(station)
                pass
        else:
            if self.debug : print "pas encore arrive"
            pass
        






In [338]:
class velo:
    def __init__(self, nom, station):
        from scipy.stats import norm
        import time 
        self.nom          = nom
        performance       = norm( loc =0.9, scale=0.2 )
        self.performance  = performance.rvs(1)[0]
        self.station      = [station]
        self.problemes    = []
        self.degradations = []
        self.performances = []
        self.debug        = False
        self.nb_km_trajet = []
        self.heures_rendu = []
        self.disponible   = True
    
    def rendu(self, station, nb_km_trajet):
        import time
        self.disponible = True
        if self.debug : print("velo %s rendu sur station %s"%(self, station))
            
        problemes       = [0] * 50
        problemes_bis   = {"pedale" : 0.2, "roue"  : 1 , "degonfle" : 0.3, "freins" : 0.05 , 0 : 0}
        problemes.extend(problemes_bis.keys())
        probleme        = random.choice(problemes)
        
        self.problemes.append(probleme)
        degradation = sum([ problemes_bis[x] for x in self.problemes if x in problemes_bis.keys()])
        
        self.performance = self.performance - self.performance*degradation
        self.performances.append(self.performance)
        self.degradations.append(degradation)
        self.nb_km_trajet.append(nb_km_trajet)
        self.station.append(station)
        self.heures_rendu.append(time.time())
        

    def __str__(self):
        return "%s"%self.nom

    def en_utilisation(self, station):
        self.disponible = False
        station.occupee = False


In [339]:
class station:
    def __init__(self, nom ):
        import random
        import time
        self.nom = nom
        self.nb_plot  = random.choice([10,15,20, 30])
        self.nb_libre = random.randint(0, self.nb_plot )
        self.nb_velos = self.nb_plot - self.nb_libre
        self.velos    = []
        noms = get_names()
        for n in range(self.nb_velos ):
            nom_velo="velo_"+"".join(random.choice(noms))
            self.velos.append(velo(nom_velo , self))
        
        print "created %s velos"%len(self.velos),
        self.debug = False
        self.avaries = []
        self.disponibilites = [("init", time.time(), self.nb_libre, self.nb_velos)]
        self.arrete = False

    def avarie_possible(self):
        import math
        import random
        import time
        avaries     = [0]*1000
        plot        = "plot mort"
        half        = "50% mort"
        network     = "reseau"
        electricite = "eteint"
        bug         = "bug"
        problemes   = { plot       : 20 ,
                       half        : 10 ,
                       network     :  5 ,
                       electricite : 30 , 
                       bug         :  3 }
        
        for k,v in problemes.iteritems():
            avaries.extend([k] * v)
            
        if plot in self.avaries : avaries.extend( [plot] * 60 )
        if half in self.avaries : avaries.extend( [half] *100 )
        
        avarie = random.choice(avaries)
        self.avaries.extend([avarie])
        if avarie == plot :
            self.nb_libre = self.nb_libre -1
        elif avarie == half:
            import math
            self.nb_libre = math.floor(self.nb_libre - (self.nb_plot / float(2)))
        elif avarie == network :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == electricite :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == bug :
            if self.nb_libre >0:
                self.nb_libre = self.nb_libre - random.randint(0, self.nb_libre)
        if self.nb_libre <0:
            self.nb_libre = 0            
        self.disponibilites.append(("%s"%avarie, time.time(), self.nb_libre, self.nb_velos))
        with open("./stations.csv", "a") as f:
            to_write = "%s, %s,%s,%s"%(self.nom, time.time(), self.nb_libre, self.nb_velos)
            f.write("%s\n"%to_write)
            
    def rendre_velo(self, velo, nb_km_trajet):
        import time
        self.avarie_possible()
        if self.nb_libre >0:
            self.velos.append(velo)
            velo.rendu(self, nb_km_trajet)
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre - 1
            self.disponibilites.append(("rendre", time.time(), self.nb_libre, self.nb_velos))
            return 1
        return 0

    def prendre_velo(self):
        """
        retourne un vélo ou 0 en cas d'erreur.
        """
        import time
        self.avarie_possible()
        if self.velos and not self.arrete :
            to_return     = self.velos.pop()
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre + 1
            self.disponibilites.append(("prendre",time.time(), self.nb_libre, self.nb_velos))
            return to_return

        return 0
    def __str__(self):
        return "station %s"%self.nom


In [357]:
def get_names(nb_lettres=3, n = 10000, prefix=""):
    """
    retourne n mot de nb_lettres avec prefix.
    """
    import itertools
    from random import shuffle
    lettres = "azertyuiopqsdfghjklmwxcvbn1234567890"
    names   = []
    for i in itertools.combinations(lettres, nb_lettres):
        name = "".join(i)
        name = prefix + name
        names.append(name)
        n = n -1
        if n <2:
            break
    shuffle(names)
    return names

In [358]:
!git init

Reinitialized existing Git repository in /Users/romain/Informatique/notebooks/.git/


In [342]:
names = get_names()
stations = [ station("".join(random.choice(names))) for i in range(50)]

created 0 velos created 1 velos created 1 velos created 7 velos created 20 velos created 3 velos created 10 velos created 11 velos created 8 velos created 1 velos created 0 velos created 4 velos created 2 velos created 4 velos created 1 velos created 13 velos created 20 velos created 16 velos created 8 velos created 7 velos created 6 velos created 1 velos created 0 velos created 5 velos created 21 velos created 23 velos created 9 velos created 14 velos created 8 velos created 8 velos created 5 velos created 9 velos created 1 velos created 11 velos created 17 velos created 6 velos created 15 velos created 17 velos created 9 velos created 4 velos created 0 velos created 11 velos created 7 velos created 1 velos created 13 velos created 2 velos created 23 velos created 14 velos created 16 velos created 14 velos


In [343]:

nb_velos = sum([len(station.velos) for station in stations])
nb_velos

427

In [344]:
names     = get_names(prefix="cycliste_")
cyclistes = [cycliste(names[n]) for n in range(nb_velos)]

In [345]:
debug = True
from time import sleep
from time import time
n = 0  
while n <3 : 
    print n,
    en_route = [cycliste for cycliste in cyclistes if     cycliste.sur_velo]
    a_pied   = [cycliste for cycliste in cyclistes if not cycliste.sur_velo]
    if debug : print "nb en route = %s"%len(en_route)
    if debug : print "nb a pied   = %s"%len(a_pied)
    for cycliste in en_route:
        cycliste.rendre_velo(random.choice(stations))
    for cycliste in a_pied:
        cycliste.prend_velo(random.choice(stations))
    n += 1

0 nb en route = 0
nb a pied   = 427
1 nb en route = 264
nb a pied   = 163
2 nb en route = 287
nb a pied   = 140


In [346]:
nb_trajet = sum([cycliste.nb_trajet for cycliste in cyclistes])
nb_trajet

136

In [347]:
import pandas as pd
tous_velos = []
velos_en_station  = [ velo for station_ in stations for velo in station_.velos  if station_.velos ]
velos_en_partance = [ cycliste_.velo for cycliste_ in cyclistes   if cycliste_.velo ]
tous_velos.extend(velos_en_station)
tous_velos.extend(velos_en_partance)
tous_velos = pd.DataFrame([x.__dict__ for x in tous_velos])
tous_velos

,debug,degradations,disponible,heures_rendu,nb_km_trajet,nom,performance,performances,problemes,station
0,False,[0],True,[1485035772.15],[11.4768456509],velo_hj3,1.085085,[1.08508530154],[0],"[station zgm, station zej]"
1,False,[],True,[],[],velo_k25,0.739844,[],[],[station v12]
2,False,[],True,[],[],velo_zkx,0.954168,[],[],[station v12]
3,False,[],True,[],[],velo_imb,0.717730,[],[],[station v12]
4,False,[],True,[],[],velo_hj1,0.739972,[],[],[station v12]
5,False,[],True,[],[],velo_zo4,1.108181,[],[],[station v12]
6,False,[],True,[],[],velo_qn0,0.776166,[],[],[station ikw]
7,False,[],True,[],[],velo_fj8,0.399757,[],[],[station ikw]
8,False,[0],True,[1485035772.42],[12.2832585761],velo_qw1,0.952947,[0.95294713153],[0],"[station zgm, station ikw]"
9,False,[],True,[],[],velo_rfl,1.091138,[],[],[station odb]


In [348]:
stations_df = pd.DataFrame([x.__dict__ for x in stations])

In [349]:
a = pd.DataFrame([x.__dict__ for x in cyclistes])